# Data Generator

Generating more realistic data

In [1]:
# helper functions
from datagen_helper import generate_pd_df

import pandas as pd

In [2]:
df_pd = generate_pd_df(20)

df_pd.to_csv('transactions-sample.csv', sep=',', index=False)
print (f'wrote to "transactions-sample.csv"')

df_pd.head(10)

wrote to "transactions-sample.csv"


,id,timestamp,mti,card_number,amount_customer,merchant_type,merchant_id,merchant_address,ref_id,amount_merchant,response_code
0,1c309b20-5074-41cd-8a46-45ebebe04cf4,2017-06-21 17:48:37.673,0100,4789625716349534,95.00,0167,81,"Star Hardware, 39 Broadway, Redwood City, UT, USA",,NaN,NaN
1,fd421703-f724-425b-99f7-436c7f2c2a8b,2017-06-21 17:48:40.425,0110,4789625716349534,95.00,0167,81,"Star Hardware, 39 Broadway, Redwood City, UT, USA",1c309b20-5074-41cd-8a46-45ebebe04cf4,92.15,00
2,f06b52bc-2944-4080-ad27-f1f8de116394,2017-09-27 14:26:07.825,0100,4885453044056032,8.19,8371,0,"Super Nursery, 998 Alameda ave, San Francisco,...",,NaN,NaN
3,11deb9b1-e9b8-4fbd-87d8-59bc3e026899,2017-09-27 14:26:08.336,0110,4885453044056032,8.19,8371,0,"Super Nursery, 998 Alameda ave, San Francisco,...",f06b52bc-2944-4080-ad27-f1f8de116394,7.94,08
4,1c8b6836-b6e8-4f3d-b3b4-a90adbbf24f8,2015-02-12 05:33:48.391,0100,4192189306372703,8.93,4245,44,"Summer Winds Dry cleaners, 611 Alameda ave, Sa...",,NaN,NaN
5,e07fe94c-ae66-48d2-9ee4-bf4ac477f2dc,2015-02-12 05:33:49.830,0110,4192189306372703,8.93,4245,44,"Summer Winds Dry cleaners, 611 Alameda ave, Sa...",1c8b6836-b6e8-4f3d-b3b4-a90adbbf24f8,8.66,13
6,919e65a3-fe22-42e6-bc79-933b1702a004,2019-08-04 14:22:19.402,0100,4532702044918856,49.18,1625,0,"First Restaurant, 178 Market St, Redwood City,...",,NaN,NaN
7,accc1ecc-037e-4428-a675-3fc8d25b170e,2019-08-04 14:22:21.934,0110,4532702044918856,49.18,1625,0,"First Restaurant, 178 Market St, Redwood City,...",919e65a3-fe22-42e6-bc79-933b1702a004,47.70,00
8,74ba886f-9be5-4c99-a892-47efb4263202,2019-06-24 21:38:04.747,0100,4122937968520189,19.18,0328,39,"Star Nursery, 332 Beach rd, Foster City, MT, USA",,NaN,NaN
9,697a72e7-0ffa-4b8f-938f-a61a7d53c333,2019-06-24 21:38:06.314,0110,4122937968520189,19.18,0328,39,"Star Nursery, 332 Beach rd, Foster City, MT, USA",74ba886f-9be5-4c99-a892-47efb4263202,18.60,00


## Create a Spark DataFrame

In [3]:
## Init Spark
import os, sys

this_dir = os.path.abspath('')
parent_dir = os.path.dirname(this_dir)
grand_parent_dir = os.path.dirname(parent_dir)
    
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from init_spark import init_spark

spark = init_spark()

Initializing Spark...
Spark found in :  /Users/sujee/apps/spark
Spark config:
	 spark.app.name=TestApp
	spark.master=local[*]
	executor.memory=2g
	spark.sql.warehouse.dir=/var/folders/lp/qm_skljd2hl4xtps5vw0tdgm0000gn/T/tmpev3yxvth
	some_property=some_value
Spark UI running on port 4040


In [4]:
import pyspark
from pyspark.sql.types import ArrayType, IntegerType, LongType, StringType, FloatType, TimestampType, StructType, StructField

my_schema = StructType([
                       StructField("id", StringType(), True),
                       StructField("timestamp", StringType(), True),
                       StructField("mti", StringType(), True),
                       StructField("card_number", StringType(), True),
                       StructField("amount_customer", FloatType(), True),
                       StructField("merchant_type", StringType(), True),
                       StructField("merchant_id", StringType(), True),
                       StructField("merchant_address", StringType(), True),
                       StructField("ref_id", StringType(), True),
                       StructField("amount_merchant", FloatType(), True),
                       StructField("response_code", StringType(), True),
                      ])


spark_df = spark.createDataFrame(df_pd, schema=my_schema)
spark_df.show(truncate=True)

+--------------------+--------------------+----+----------------+---------------+-------------+-----------+--------------------+--------------------+---------------+-------------+
|                  id|           timestamp| mti|     card_number|amount_customer|merchant_type|merchant_id|    merchant_address|              ref_id|amount_merchant|response_code|
+--------------------+--------------------+----+----------------+---------------+-------------+-----------+--------------------+--------------------+---------------+-------------+
|1c309b20-5074-41c...|2017-06-21 17:48:...|0100|4789625716349534|           95.0|         0167|         81|Star Hardware, 39...|                    |            NaN|          NaN|
|fd421703-f724-425...|2017-06-21 17:48:...|0110|4789625716349534|           95.0|         0167|         81|Star Hardware, 39...|1c309b20-5074-41c...|          92.15|           00|
|f06b52bc-2944-408...|2017-09-27 14:26:...|0100|4885453044056032|           8.19|         8371|     